In [326]:
using CSV, DelimitedFiles, DataFrames, Missings, XLSX
using LinearAlgebra, Statistics, Optim, StatsBase
using Random, Distributions, LoopVectorization
using GeneNetworkAPI, Downloads
using BenchmarkTools

In [2]:
using Plots

In [3]:
pwd()

"/home/zyu20/git/BulkLMM_Analyses/HS-Palmer"

In [4]:
local_path = "../../BulkLMM.jl/src";

In [327]:
include(joinpath(local_path, "kinship.jl"));
include(joinpath(local_path, "util.jl"));
include(joinpath(local_path, "wls.jl"));
include(joinpath(local_path, "lmm.jl"));
include(joinpath(local_path, "gridbrent.jl"));
include(joinpath(local_path, "transform_helpers.jl"));
include(joinpath(local_path, "scan.jl"));
include(joinpath(local_path, "bulkscan_helpers.jl"));
include(joinpath(local_path, "bulkscan.jl"));
include(joinpath(local_path, "readData.jl"));
include(joinpath(local_path, "../plot_utils/visuals_utils.jl"));
include(joinpath(local_path, "analysis_helpers/single_trait_analysis.jl"));
include("../../BigRiver_util_code/src/kinship_utils.jl");

## Omics traits:

In [185]:
@time omics_pheno_df = CSV.read("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Rat-PL-RSeq-0818_nomissing.csv", DataFrame);

  1.337354 seconds (1.11 M allocations: 84.567 MiB, 3.94% gc time)


In [10]:
names(omics_pheno)[1:6]

6-element Vector{String}:
 "id"
 "ENSRNOG00000000001"
 "ENSRNOG00000000007"
 "ENSRNOG00000000008"
 "ENSRNOG00000000009"
 "ENSRNOG00000000010"

In [11]:
sample_ids = omics_pheno[:, 1];

## HS Genotypes：

Takes too long...

In [36]:
# @time hs_geno = parse_geno("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true.geno")

In [37]:
open("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true.geno") do io
           readline(io) # throw out the first line
           readline(io)
           # readlines(io)
end

"@name:HSNIH-Palmer"

In [118]:
count = 0;
test_line = readline("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true.geno")

for line in eachline("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true.geno")
    
    count = count + 1;
    println(line)
    
    test_line = line
    
    if count == 8
        break
    end
    
end

@type:riset
@name:HSNIH-Palmer
@mat:0
@pat:2
#heterozygous, optional, default is \"H\"
@het:1
#Unknown, optional, default is \"U\"
@unk:U


In [48]:
geno_colnames = split(test_line, '\t')

6151-element Vector{SubString{String}}:
 "Chr"
 "Locus"
 "cM"
 "Mb"
 "0007A0008B"
 "0007A00024"
 "0007A000DB"
 "0007A001C5"
 "0007A0059F"
 "0007A00263"
 "0007A00670"
 "0007A00716"
 "0007A01A7C"
 ⋮
 "0007929889"
 "0007929894"
 "0007929913"
 "0007929918"
 "0007929924"
 "0007929944"
 "0007929945"
 "0007929959"
 "0007929963"
 "0007929965"
 "0007929994"
 "0007929999"

In [100]:
size(geno_colnames)

(6151,)

In [55]:
@time cols_to_extract_from_fullgeno = map(x -> x in sample_ids, geno_colnames);

  0.085747 seconds (553.54 k allocations: 33.284 MiB, 59.11% compilation time)


In [56]:
sum(cols_to_extract_from_fullgeno)

80

In [102]:
cols_ids_in_geno = findall(cols_to_extract_from_fullgeno .== 1) 

80-element Vector{Int64}:
 2227
 2282
 3571
 4113
 4669
 4691
 4697
 4701
 4710
 4716
 4730
 4736
 4742
    ⋮
 5355
 5370
 5372
 5377
 5385
 5496
 5497
 5507
 5532
 5536
 5540
 5541

In [130]:
#=
@time begin
    myfile = open("my_genofile.txt", "w")
    row_count = 0
    for line in eachline("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true.geno")

        row_count = row_count + 1;

        if row_count >= 9
            col_count = 0;
            words_in_curr_line = split(line, '\t');
            for word in words_in_curr_line
                col_count = col_count + 1;
                if col_count in cols_ids_in_geno
                    to_write = word * "\t";
                    write(myfile, to_write);
                end
            end
            write(myfile, "\n")
        end
    end

    close(myfile)
end
=#

In [129]:
@time omics_geno = readdlm("my_genofile.txt", '\t')[2:end, 1:(end-1)]

 70.605341 seconds (192.58 M allocations: 5.088 GiB, 83.14% gc time, 0.06% compilation time)


134918×80 Matrix{Any}:
 2        2        1        2        …  2        1        2        2
 2        2        2        2           2        2        2        2
 2        2        2        1           2        2        2        2
 2        2        2        1           2        2        2        2
 2        2        2        1           2        2        2        1
 2        2        2        2        …  2        2        1        2
 2        2        2        2           2        2        1        2
 0.997    1.976    0.991    1.979       1.979    1.979    1.979    0.007
 1.941    1.559    1.006    1.994       1.993    1.994    1.993    1.998
 1.978    1.992    1.997    1.994       1.993    1.993    1.993    1.997
 0.031    0.39     0.99298  0.008    …  0.01     0.009    0.01     0.003
 1.997    1.997    1.998    1.997       1.997    1.997    1.997    1.998
 1.997    1.997    1.006    1.997       1.997    1.997    1.997    1.998
 ⋮                                   ⋱                  

In [146]:
NA_entries = findall(x -> typeof(x) == SubString{String}, omics_geno)

1494-element Vector{CartesianIndex{2}}:
 CartesianIndex(21302, 1)
 CartesianIndex(21316, 1)
 CartesianIndex(35852, 1)
 CartesianIndex(35855, 1)
 CartesianIndex(78338, 1)
 CartesianIndex(78340, 1)
 CartesianIndex(78341, 1)
 CartesianIndex(78363, 1)
 CartesianIndex(95701, 1)
 CartesianIndex(95704, 1)
 CartesianIndex(95731, 1)
 CartesianIndex(95812, 1)
 CartesianIndex(21302, 2)
 ⋮
 CartesianIndex(95776, 80)
 CartesianIndex(95777, 80)
 CartesianIndex(95779, 80)
 CartesianIndex(95781, 80)
 CartesianIndex(95782, 80)
 CartesianIndex(95784, 80)
 CartesianIndex(95787, 80)
 CartesianIndex(95812, 80)
 CartesianIndex(109662, 80)
 CartesianIndex(124557, 80)
 CartesianIndex(124558, 80)
 CartesianIndex(130075, 80)

There are still some missings coded as "NA"'s...

In [144]:
omics_geno[21302, 1]

"NA"

In [145]:
omics_geno[130075, 80]

"NA"

In [154]:
NA_markers = zeros(length(NA_entries));

for i in 1:length(NA_entries)
    NA_markers[i] = NA_entries[i][1];
end

NA_markers = trunc.(Int64, NA_markers);

In [155]:
NA_markers

1494-element Vector{Int64}:
  21302
  21316
  35852
  35855
  78338
  78340
  78341
  78363
  95701
  95704
  95731
  95812
  21302
      ⋮
  95776
  95777
  95779
  95781
  95782
  95784
  95787
  95812
 109662
 124557
 124558
 130075

In [210]:
omics_geno_nomissing = omics_geno[map(x -> !(x in NA_markers), collect(1:size(omics_geno, 1))), :]';

In [166]:
# writedlm("/home/zyu20/shareddata/HSNIH-Palmer/HSNIH-Palmer_true_omics_geno_nomissing.txt", omics_geno_nomissing, '\t')

## Load BulkLMM functions:

In [220]:
include(joinpath(local_path, "kinship.jl"));
include(joinpath(local_path, "util.jl"));
include(joinpath(local_path, "wls.jl"));
include(joinpath(local_path, "lmm.jl"));
include(joinpath(local_path, "gridbrent.jl"));
include(joinpath(local_path, "transform_helpers.jl"));
include(joinpath(local_path, "scan.jl"));
include(joinpath(local_path, "bulkscan_helpers.jl"));
include(joinpath(local_path, "bulkscan.jl"));
include(joinpath(local_path, "readData.jl"));
include(joinpath(local_path, "../plot_utils/visuals_utils.jl"));
include(joinpath(local_path, "analysis_helpers/single_trait_analysis.jl"));
include("../../BigRiver_util_code/src/kinship_utils.jl");
include("../../BigRiver_util_code/src/run_gemma_utils.jl");

In [243]:
omics_geno_nomissing = float.(omics_geno_nomissing)./2.0;

In [212]:
size(omics_geno_nomissing)

(80, 134681)

In [213]:
quantile(vec(omics_geno_nomissing), collect(0.1:0.1:1.0))

10-element Vector{Float64}:
 0.0
 0.003
 0.005
 0.965
 1.0
 1.0
 1.668
 1.994
 1.998
 2.0

In [187]:
omics_pheno = omics_pheno_df[1:end, 2:end] |> Matrix{Float64};

In [181]:
grid_loose = collect(0.0:0.05:0.95);

In [200]:
mapslices(x -> sum(x), omics_pheno; dims = 1)

1×32623 Matrix{Float64}:
 295.64  970.256  121.756  -3.3276  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [203]:
nonzero_traits_to_keep = findall(vec(mapslices(x -> sum(x), omics_pheno; dims = 1)) .!== 0.0)

18416-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 18405
 18406
 18407
 18408
 18409
 18410
 18411
 18412
 18413
 18414
 18415
 18416

In [205]:
omics_pheno_nozeros = omics_pheno[:, nonzero_traits_to_keep];

In [206]:
size(omics_pheno_nozeros)

(80, 18416)

In [365]:
@time maf_omics_geno = vec(mean(omics_geno_nomissing; dims = 1));

  0.011095 seconds (10 allocations: 1.028 MiB)


In [373]:
to_keep = map(x -> (x < 0.95) & (x > 0.05), maf_omics_geno);

In [375]:
omics_geno_nomissing_filtered = omics_geno_nomissing[:, to_keep];

In [376]:
@time omics_kinship = calcKinship(omics_geno_nomissing_filtered)

 24.348417 seconds (31.60 k allocations: 13.847 GiB, 65.29% gc time)


80×80 Matrix{Float64}:
 1.0       0.647764  0.638984  0.640744  …  0.633285  0.639396  0.662753
 0.647764  1.0       0.634283  0.63157      0.627148  0.62953   0.65366
 0.638984  0.634283  1.0       0.650255     0.648052  0.635979  0.646126
 0.640744  0.63157   0.650255  1.0          0.635719  0.652099  0.640436
 0.637076  0.633313  0.639172  0.646493     0.629846  0.64328   0.64063
 0.676449  0.645656  0.653192  0.656735  …  0.638355  0.640651  0.642787
 0.641324  0.631591  0.645213  0.649611     0.631336  0.638216  0.640156
 0.634784  0.643335  0.631772  0.646329     0.647447  0.640572  0.656369
 0.639598  0.619243  0.641125  0.640551     0.634354  0.6276    0.644544
 0.653657  0.646809  0.638325  0.637831     0.649555  0.636434  0.649134
 0.644224  0.64628   0.64461   0.644455  …  0.636753  0.652057  0.633401
 0.62735   0.639572  0.642622  0.627293     0.63297   0.627297  0.652782
 0.646551  0.653715  0.651631  0.658093     0.635651  0.63378   0.637987
 ⋮                            

In [377]:
findall(var(omics_geno_nomissing_filtered; dims = 1) .== 0.0)

CartesianIndex{2}[]

In [341]:
omics_geno_nomissing_filtered

80×134073 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.4985  …  1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.988      1.0  1.0  0.999   0.0065  1.0
 0.5  1.0  1.0  1.0  1.0  1.0  1.0  0.4955     1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  0.5  0.5  0.5  1.0  1.0  0.9895     1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0035     0.5  0.5  0.498   0.0455  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.9895  …  1.0  1.0  0.999   0.0065  1.0
 0.5  0.5  1.0  1.0  1.0  0.5  1.0  0.4955     1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.4985     1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.004      1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.9895     1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.4985  …  1.0  1.0  0.999   0.0065  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.4985     1.0  0.5  0.9975  0.007   1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0035     1.

In [347]:
@time bulkscan_null_grid_results = bulkscan_null_grid(omics_pheno_nozeros, omics_geno_nomissing_filtered, omics_kinship, grid_loose);

 59.806108 seconds (7.78 M allocations: 39.142 GiB, 40.36% gc time)


In [360]:
@time bulkscan_alt_grid_results = bulkscan_alt_grid(omics_pheno_nozeros, omics_geno_nomissing_filtered, omics_kinship, grid_loose);

1709.444709 seconds (38.38 M allocations: 1.556 TiB, 24.61% gc time, 0.02% compilation time)


In [362]:
bulkscan_alt_grid_results.h2_panel

134073×18416 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

## GEMMA:

In [ ]:
gemma = "/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64"

"/home/zyu20/Softwares/gemma-0.98.5-linux-static-AMD64"

#### Issues with data found so far:

- Identical genotype probabilities at some markers for all samples;